In [1]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from math import pi

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time 
from operator import length_hint 

from selenium.common.exceptions import NoSuchElementException

In [2]:
browser = webdriver.Chrome('Documents/Driver/chromedriver')    

browser.get('https://www.linkedin.com/uas/login')

file = open('Documents/Config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

In [3]:
elementID.submit()

In [5]:
visitingcompany = '/search/results/people/?currentCompany=%5B"33223"%5D&'
fulllink = 'https://www.linkedin.com' + visitingcompany
browser.get(fulllink)

In [6]:
profilesqueued = []

def getnewprofile(soup, profilesqueued):
    ProfilesID = []
    pav = soup.find('div', {'class':'ph0 pv2 artdeco-card mb2'})
    profilelink = soup.findAll('a', {'aria-hidden':'false'})
    for links in profilelink:
        userLink = links.get('href')
        profilesqueued.append(userLink)
    return (userLink)

In [7]:
# This code allows scraping all linkedIn profiles as it is flipping to the next page
try:
    x = 0
    while True:
        browser.execute_script("window.scrollTo(0, 10000);")
        time.sleep(1)
        (getnewprofile(BeautifulSoup(browser.page_source), profilesqueued))
        nextpage = browser.find_element_by_xpath("//button[@class='artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view']").click()
        time.sleep(1)
        x += 1
except NoSuchElementException:
        pass

In [18]:
columns=['employee_name', 'Current_title', 'company_name1','company_name2']
df = pd.DataFrame(columns=columns)

In [19]:
x = 1
try:
    while x < len(profilesqueued):
        browser.get(profilesqueued[x])

        Name = []
        Currenttitle = []
        Company1 = []
        Company2 = []

        source = browser.page_source
        soup = BeautifulSoup(source, 'lxml')
        
        #Get Employee Name
        
        employee_name_section_div = soup.find('div',{'class':'pv-text-details__left-panel mr5'})
        employee_name = employee_name_section_div.find('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'}).get_text().strip()

        Name.append(employee_name)
        
        experience_section = browser.find_element(By.XPATH, '//*[@id="experience-section"]/header/h2')
        browser.execute_script("arguments[0].scrollIntoView();",experience_section)
        
        
        #Get Current Company Title
        
        Current_title_li = soup.findAll('li',{'class':'pv-entity__position-group-pager pv-profile-section__list-item ember-view'})
        try:
            Current_title = Current_title_li[0].find('h3',{'class':'t-16 t-black t-bold'}).get_text().strip()
        except:
            Current_title=None

        Currenttitle.append(Current_title)
        
        
        #Get Previous Company Name 1

        company_name_li = soup.findAll('li',{'class':'pv-entity__position-group-pager pv-profile-section__list-item ember-view'})
        try:
            company_name1 = company_name_li[1].find('p',{'class':'pv-entity__secondary-title t-14 t-black t-normal'}).get_text().strip()
        except:
            company_name1=None

        Company1.append(company_name1)
        
        
        #Get Previous Company Name 1

        company_name_li = soup.findAll('li',{'class':'pv-entity__position-group-pager pv-profile-section__list-item ember-view'})
        try:
            company_name2 = company_name_li[2].find('p',{'class':'pv-entity__secondary-title t-14 t-black t-normal'}).get_text().strip()
        except:
            company_name2 = None

        Company2.append(company_name2)

        
        time.sleep(1)

        
        df=df.append({ 'employee_name' : employee_name, 'Current_title' : Current_title ,'company_name1' : company_name1, 'company_name2' : company_name2 } ,ignore_index=True)
        x += 1
except NoSuchElementException:
    pass 
print ("Scraping Stopped!!!")

Scraping Stopped!!!


In [154]:
df.to_excel('Desktop/Excel Bot Output/LinkedIn Profiles with Job Title.xls', index = False, header=True)


<ipython-input-154-60e6ee5871b5>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('Desktop/Excel Bot Output/LinkedIn Profiles with Job Title.xls', index = False, header=True)


In [22]:
del df

In [23]:
df

NameError: name 'df' is not defined